# Variant calling analysis Killswitch circuit 

Defining variants (SNPs, insertions and deletions) for a set of HiSeq samples exploring the variability in populations of *M. pneumoniae* transformed with a killswitch cassette (C5 samples) or two (C20 samples); sequencing performed at different selection passages (2, 3, 15; 2 replicates in each condition and cassette) and a pair of replicates at passage 3 were the cassette was induced by IPTG.

**Authors:** [Samuel Miravet-Verde](mailto:samuel.miravet@crg.eu) and [Alicia Broto](mailto:alicia.broto@crg.eu) 

**Last update:** 14/06/2021

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

import sys, os
import glob
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scripts
from collections import Counter
from Bio import SeqIO
from Bio.SeqFeature import SeqFeature, FeatureLocation

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# General info about the genome references and the location of the cassettes in the genomes
seq_record5 = SeqIO.read('./data/C5_refSeq.gb', "genbank")
seq_record20 = SeqIO.read('./data/C20_refSeq.gb', "genbank")
genome_C5 = str(seq_record5.seq)
genome_C20 = str(seq_record5.seq)

cass_par = [565510, 572124] # Present in C5 and C20
cass_3b = [780850, 787445]  # Only in C20

# Annotation information to work faster with dataframes
ncbi = pd.read_csv('./data/mpn_annotation.csv', sep='\t', header=None) # Genome annotation
ncbi.columns = ['gene', 'start', 'end', 'strand']
ncbi.set_index('gene', inplace=True)
gold = pd.read_csv('./data/goldsets.csv', sep='\t')  # Gold set 

posN = []
posE = []
for gene, cat in zip(gold['gene'], gold['class']):
    if cat=='E':
        posE += range(ncbi.loc[gene][0], ncbi.loc[gene][1]+1)
    else:
        posN += range(ncbi.loc[gene][0], ncbi.loc[gene][1]+1)
        
# Define the different set of positions in the genome were we will perform the analysis
posN = set(posN)
posE = set(posE)
cas1 = set(range(cass_par[0], cass_par[1]+1))
cas2 = set(range(cass_3b[0], cass_3b[1]+1))
cas12 = cas1.union(cas2)
geno = set(range(1, 816395)).difference(cas1.union(cas2))

## 1. Map variations

We will be using [Snippy](https://github.com/tseemann/snippy) for the first steps related to sequence mapping. To run it in parallel for all the samples we create a input file:

In [4]:
### 1. Make a multifile to map with snippy-multi.
files = glob.glob('../data/C[5|20]*/*_R1_*') # The gre expression refers to the raw sequencing files located in data (this has to point to the directory where fastq files are stored)

fo1 = open('./tmp_files/snippy_input1.txt', 'w')
fo2 = open('./tmp_files/snippy_input2.txt', 'w')
for f in files:
    ide = f.split('/')[2:]
    if 'C5' in f:
        ide = ide[0].replace('C5', '')+'_'+'_'.join(ide[1].split('_')[:-3])
        fo1.write('{}\t{}\t{}\n'.format(ide, f, f.replace('_R1_', '_R2_')))
    else:
        ide = ide[0].replace('C20', '')+'_'+'_'.join(ide[1].split('_')[:-3])
        fo2.write('{}\t{}\t{}\n'.format(ide, f, f.replace('_R1_', '_R2_')))
fo1.close()
fo2.close()
files

['../data/C20p3/B_S1_L001_R1_001.fastq.gz',
 '../data/C20p3/B_S2_L001_R1_001.fastq.gz',
 '../data/C5p3/D_S2_L001_R1_001.fastq.gz',
 '../data/C5p3/D_S3_L001_R1_001.fastq.gz',
 '../data/C5p3IPTG/A_S1_L001_R1_001.fastq.gz',
 '../data/C5p3IPTG/A_S1b_L001_R1_001.fastq.gz',
 '../data/C20p15/7_S3_L001_R1_001.fastq.gz',
 '../data/C20p15/7_S5_L001_R1_001.fastq.gz',
 '../data/C5p15/F_S5_L001_R1_001.fastq.gz',
 '../data/C5p15/F_S1_L001_R1_001.fastq.gz',
 '../data/C5p2/2_S2_L001_R1_001.fastq.gz',
 '../data/C5p2/2_S3_L001_R1_001.fastq.gz',
 '../data/C5p2/2_S1_L001_R1_001.fastq.gz',
 '../data/C20p2/6_S2_L001_R1_001.fastq.gz',
 '../data/C20p2/6_S4_L001_R1_001.fastq.gz']

Now run the command in the snippy environment:

```bash
conda activate snippy
snippy-multi ./data/snippy_input.txt --ref ./data/C5_refSeq.gb --cpus 4 --report >  ./snippy_run.sh'
sh ./snippy_run.sh
```

Run freebayes in a permissive manner. This command will create a directory with the name of the sample were all the files will be stored. This is expected to be run in the terminal copy-pasting the printed command. It can also be run changing the print to `os.system` to execute it for each file:

In [5]:
# Listed samples in order
order1 = ['p2_2_S1','p2_2_S2','p2_2_S3','p3_D_S2', 'p3_D_S3','p15_F_S1','p15_F_S5', 'p3IPTG_A_S1', 'p3IPTG_A_S1b']
order2 = ['p2_6_S2','p2_6_S4','p3_B_S1','p3_B_S2','p15_7_S3','p15_7_S5']

In [6]:
# freebayes-parallel reference/ref.txt 4 -p 2 -P 0 -C 2 -F 0.05 --min-coverage 10 --min-repeat-entropy 1.0 -q 13 -m 60 --strict-vcf   -f reference/ref.fa snps.bam > snps.raw.vcf
for fil in glob.glob('../p*/*.bam'):
    # again this has to point to directories were snipy output is stored
    ide = fil.split('/')[1]
    #print('freebayes-parallel {}/reference/ref.txt 4 -p 1 -P 0 -C 1 -F 0.01 -q 13 -m 60 --strict-vcf -f {}/reference/ref.fa {} > filter10/{}.raw.vcf'.format(ide, ide, fil, ide))
    #print('freebayes-parallel {}/reference/ref.txt 4 -p 1 -P 0 -C 2 -F 0.05 -q 13 -m 60 --strict-vcf -f {}/reference/ref.fa {} > filter50/{}.raw.vcf'.format(ide, ide, fil, ide))
    if ide in order1:
        print('freebayes-parallel {}/reference/ref.txt 4 -p 1 -P 0 -C 1 -F 0.001 -q 13 -m 60 --strict-vcf -f {}/reference/ref.fa {} > filter1/{}.raw.vcf'.format(ide, ide, fil, ide))
    else:
        print('freebayes-parallel {}/reference/ref.txt 4 -p 1 -P 0 -C 1 -F 0.001 -q 13 -m 60 --strict-vcf -f {}/reference/ref.fa {} > filter2/{}.raw.vcf'.format(ide, ide, fil, ide))
        


freebayes-parallel p3_D_S2/reference/ref.txt 4 -p 1 -P 0 -C 1 -F 0.001 -q 13 -m 60 --strict-vcf -f p3_D_S2/reference/ref.fa ../p3_D_S2/snps.bam > filter1/p3_D_S2.raw.vcf
freebayes-parallel p2_6_S2/reference/ref.txt 4 -p 1 -P 0 -C 1 -F 0.001 -q 13 -m 60 --strict-vcf -f p2_6_S2/reference/ref.fa ../p2_6_S2/snps.bam > filter2/p2_6_S2.raw.vcf
freebayes-parallel p2_2_S1/reference/ref.txt 4 -p 1 -P 0 -C 1 -F 0.001 -q 13 -m 60 --strict-vcf -f p2_2_S1/reference/ref.fa ../p2_2_S1/snps.bam > filter1/p2_2_S1.raw.vcf
freebayes-parallel p15_7_S5/reference/ref.txt 4 -p 1 -P 0 -C 1 -F 0.001 -q 13 -m 60 --strict-vcf -f p15_7_S5/reference/ref.fa ../p15_7_S5/snps.bam > filter2/p15_7_S5.raw.vcf
freebayes-parallel p3IPTG_A_S1/reference/ref.txt 4 -p 1 -P 0 -C 1 -F 0.001 -q 13 -m 60 --strict-vcf -f p3IPTG_A_S1/reference/ref.fa ../p3IPTG_A_S1/snps.bam > filter1/p3IPTG_A_S1.raw.vcf
freebayes-parallel p3_D_S3/reference/ref.txt 4 -p 1 -P 0 -C 1 -F 0.001 -q 13 -m 60 --strict-vcf -f p3_D_S3/reference/ref.fa ../p3_

To get the total number of mapped reads:

```python
for fil in glob.glob('./p*/*.bam'):
    ide = fil.split('/')[1]
    # get the total number of reads of a BAM file (may include unmapped and duplicated multi-aligned reads)
    print('samtools view -c {} >> total_readcount.txt'.format(fil))
    print('samtools view -c {} -F 260 >> total_readcount_mapped.txt'.format(fil))
```

Finally we annotate the effect of the variants using SnpEff. This requires to add the custom annotation to the sources, specifying that the translation code is set to 4 for *M. pneumoniae*.

1. Download SnpEff and edited the config (in software_crg/SnpEff) to include the C5 and C20 refseq references
2. Build the custom db:
```bash
java -jar snpEff.jar build -genbank -v C5ali
java -jar snpEff.jar build -genbank -v C20ali
```
3. Annotate vcf files in bash (in directory of the software)
```bash
for fil in `ls ./filter1/p*.vcf`; do java -Xmx8g -jar snpEff.jar C5ali $fil > $fil.ann; done
for fil in `ls ./filter2/p*.vcf`; do java -Xmx8g -jar snpEff.jar C20ali $fil > $fil.ann; done
```

## 2. Data loading 

We will parse the variations to keep the columns of interest to perform the exploratory analysis. Dataframes include the following information (by column header):
- SAMPLE: sample identifier includes information about the passage (e.g. p3IPTG is the passage 3 induced)
- PASS: passage (2,3 or 15)
- POS: genome loci (base pair position) where the variant is found
- QUAL: estimate of the probability that there is a polymorphism at the loci described by the record
- TOT: total number of reads covering a loci
- REFN: total number of reads covering a loci matching the reference
- ALTN: number of reads presenting a variant
- REF: reference sequence
- ALT: alternative sequence
- EFF: potential effect of the variant as given by snpeff
- IMPACT: potential impact of the variant, can be LOW (synonimous mutations), MODERATE (missense), HIGH (non-synonimous mutations, start lost, stop lost), or MODIFIER (when it occurs in an intergenic region)
- AFF: gene affected
- MUT: mutation (in nucleotide if not coding in amino acid if coding)
- ANN_TYPE: annotation of the loci, can be intergenic, gene, or cassette (covering the principal elements in the cassette)


In [7]:
# This will concat all the annotated vcf files keeping the fields of interest
snpcalls1 = scripts.parse_variations('./filter1/*.raw.vcf.ann', order1, genome=5, fname='./results/data1.pickle')
snpcalls2 = scripts.parse_variations('./filter2/*.raw.vcf.ann', order2, genome=20, fname='./results/data2.pickle')
snpcalls1


Loading data for genome 5
Loading data for genome 20


,SAMPLE,PASS,POS,QUAL,TOT,REFN,ALTN,FRAC,REF,ALT,EFF,IMPACT,AFF,MUT,ANN_TYPE
1,p2_2_S1,2,70,4.927350e-15,60,58,2,3.448276,C,T,upstream_gene_variant,MODIFIER,MPN001,c.-622C>T,intergenic
2,p2_2_S1,2,149,0.000000e+00,75,74,1,1.351351,T,C,upstream_gene_variant,MODIFIER,MPN001,c.-543T>C,intergenic
3,p2_2_S1,2,150,2.263910e-15,69,68,1,1.470588,A,T,upstream_gene_variant,MODIFIER,MPN001,c.-542A>T,intergenic
4,p2_2_S1,2,154,0.000000e+00,62,61,1,1.639344,T,C,upstream_gene_variant,MODIFIER,MPN001,c.-538T>C,intergenic
5,p2_2_S1,2,176,3.420820e-15,58,57,1,1.754386,TAAT,CAAC,upstream_gene_variant,MODIFIER,MPN001,c.-516_-513delTAATinsCAAC,intergenic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
508235,p3IPTG_A_S1b,18,822775,3.631530e-15,171,170,1,0.588235,AACGT,GACGC,missense_variant,MODERATE,MPN688,p.Asn44Ser,gene
508236,p3IPTG_A_S1b,18,822787,1.152910e-15,175,173,2,1.156069,A,G,synonymous_variant,LOW,MPN688,p.Gly41Gly,gene
508237,p3IPTG_A_S1b,18,822830,3.857310e-15,165,164,1,0.609756,T,G,missense_variant,MODERATE,MPN688,p.Lys27Thr,gene
508238,p3IPTG_A_S1b,18,822867,0.000000e+00,172,171,1,0.584795,TTTTT,CTTTC,missense_variant,MODERATE,MPN688,p.Thr15Ala,gene


In [12]:
for i in list(Counter(snpcalls1['ANN_TYPE']).keys()):
    print(i)

intergenic
gene
cassette


In [8]:
snpcalls1
# Save supplementary
sup1 = './results/supplementary1_variantcalling.xlsx'
if not os.path.isfile(sup1):
    writer = pd.ExcelWriter(sup1, engine='xlsxwriter')
    for c, data in zip(['C5_', 'C20_'], [snpcalls1, snpcalls2]):
        for sample in set(data['SAMPLE']):
            subdata = data[data['SAMPLE']==sample].copy()
            subdata.to_excel(writer, sheet_name=c+sample)
    writer.save()


,POS,REF,ALT,QUAL,TOT,REFN,ALTN,ALTS,FRAC,SAMPLE,P,EFF
0,70,C,T,4.927350e-15,60,58,2,2,3.333333,p2_2_S1,2,T|upstream_gene_variant|MODIFIER|dnaN|MPN001|t...
1,149,T,C,0.000000e+00,75,74,1,1,1.333333,p2_2_S1,2,C|upstream_gene_variant|MODIFIER|dnaN|MPN001|t...
2,150,A,T,2.263910e-15,69,68,1,1,1.449275,p2_2_S1,2,T|upstream_gene_variant|MODIFIER|dnaN|MPN001|t...
3,154,T,C,0.000000e+00,62,61,1,1,1.612903,p2_2_S1,2,C|upstream_gene_variant|MODIFIER|dnaN|MPN001|t...
4,176,TAAT,CAAC,3.420820e-15,58,57,1,1,1.724138,p2_2_S1,2,CAAC|upstream_gene_variant|MODIFIER|dnaN|MPN00...
...,...,...,...,...,...,...,...,...,...,...,...,...
508435,822775,AACGT,GACGC,3.631530e-15,171,170,1,1,0.584795,p3IPTG_A_S1b,18,GACGC|missense_variant|MODERATE|soj|MPN688|tra...
508436,822787,A,G,1.152910e-15,175,173,2,2,1.142857,p3IPTG_A_S1b,18,G|synonymous_variant|LOW|soj|MPN688|transcript...
508437,822830,T,G,3.857310e-15,165,164,1,1,0.606061,p3IPTG_A_S1b,18,G|missense_variant|MODERATE|soj|MPN688|transcr...
508438,822867,TTTTT,CTTTC,0.000000e+00,172,171,1,1,0.581395,p3IPTG_A_S1b,18,CTTTC|missense_variant|MODERATE|soj|MPN688|tra...


In [106]:
# Example:
## Notice the information is grouped by position, we will deal with this in section 3. 
snpcalls1[snpcalls1['POS'].isin(range(572044, 572066))]

,POS,REF,ALT,QUAL,TOT,REFN,ALTN,ALTS,FRAC,SAMPLE,P,EFF
18805,572048,TTAT,"TAAC,CTAG",0.000000e+00,112,110,2,"1,1",1.785714,p2_2_S1,2,CTAG|upstream_gene_variant|MODIFIER|Gene_56771...
18806,572054,A,G,0.000000e+00,109,108,1,1,0.917431,p2_2_S1,2,G|upstream_gene_variant|MODIFIER|Gene_567711_5...
60084,572047,T,A,0.000000e+00,72,71,1,1,1.388889,p2_2_S2,2,A|upstream_gene_variant|MODIFIER|Gene_567711_5...
60085,572049,TAT,AAC,0.000000e+00,78,77,1,1,1.282051,p2_2_S2,2,AAC|upstream_gene_variant|MODIFIER|Gene_567711...
60086,572055,TTCTAAAT,"GTCTGAAT,TTCCAAAC,CTCTAAAT",0.000000e+00,79,76,3,"1,1,1",3.797468,p2_2_S2,2,CTCTAAAT|upstream_gene_variant|MODIFIER|Gene_5...
60087,572063,A,"G,T",6.238460e-15,80,78,2,"1,1",2.500000,p2_2_S2,2,G|upstream_gene_variant|MODIFIER|Gene_567711_5...
117712,572052,TAATT,AAATA,1.808350e-15,261,260,1,1,0.383142,p2_2_S3,2,AAATA|upstream_gene_variant|MODIFIER|Gene_5677...
117713,572058,T,C,0.000000e+00,257,256,1,1,0.389105,p2_2_S3,2,C|upstream_gene_variant|MODIFIER|Gene_567711_5...
207709,572051,TTA,CTT,1.529410e-14,613,612,1,1,0.163132,p3_D_S2,3,CTT|upstream_gene_variant|MODIFIER|Gene_567711...
207710,572058,TAAAT,AAAAC,1.225020e-14,619,618,1,1,0.161551,p3_D_S2,3,AAAAC|upstream_gene_variant|MODIFIER|Gene_5677...


In [11]:
snpcalls2[(snpcalls2['FRAC']>=5) & (snpcalls2['FRAC']<100)].sort_values('FRAC')

,POS,REF,ALT,QUAL,TOT,REFN,ALTN,ALTS,FRAC,SAMPLE,P,EFF
418107,826526,T,C,2.932180e-14,160,152,8,8,5.000000,p15_7_S5,15,C|stop_lost|HIGH|cysA|MPN685|transcript|MPN685...
396742,603330,ATTAATAA,"GTTAGTAG,ACTAATAA,ATTAAAAA,ATTAATAG,ATTAATAT",5.831640e-15,100,95,5,"1,1,1,1,1",5.000000,p15_7_S5,15,GTTAGTAG|upstream_gene_variant|MODIFIER|P02_or...
100139,183736,CAACCTCACCCC,"CGACGTCAGCCC,GAGGCTCACCCC,CAGCCTCAGCGC,CAACGTC...",1.763150e-15,80,76,4,"1,1,1,1",5.000000,p2_6_S4,2,GAGGCTCACCCC|missense_variant|MODERATE|P1|MPN1...
394018,572827,T,G,4.776910e-15,20,19,1,1,5.000000,p15_7_S5,15,G|missense_variant|MODERATE|H08_orf157a|MPN463...
100420,187399,A,C,5.032580e-15,100,95,5,5,5.000000,p2_6_S4,2,C|missense_variant|MODERATE|orf6|MPN142|transc...
...,...,...,...,...,...,...,...,...,...,...,...,...
225401,780968,AGTCC,CGTCC,6.689500e+03,214,2,212,209,99.065421,p3_B_S1,3,CGTCC|upstream_gene_variant|MODIFIER|E09_orf30...
413979,780968,AGTCC,CGTCC,3.700950e+03,118,1,117,116,99.152542,p15_7_S5,15,CGTCC|upstream_gene_variant|MODIFIER|E09_orf30...
163301,780968,AGTCC,CGTCC,4.857580e+03,156,1,155,151,99.358974,p2_6_S4,2,CGTCC|upstream_gene_variant|MODIFIER|E09_orf30...
163413,786151,TCT,TTT,5.734500e+03,182,1,181,178,99.450549,p2_6_S4,2,TTT|synonymous_variant|LOW|Gene_785567_786226|...


## 3. Effect of the mutations

Deconvolute alternative and total read count to evaluate the population fraction and impact of each variant within the cassette.

In [118]:
correspondance(20, positions=cas1)

{0: [SeqFeature(FeatureLocation(ExactPosition(565549), ExactPosition(566678), strand=-1), type='CDS'),
  'CDS'],
 1: [SeqFeature(FeatureLocation(ExactPosition(566678), ExactPosition(566721), strand=-1), type='promoter'),
  'promoter'],
 2: [SeqFeature(FeatureLocation(ExactPosition(567711), ExactPosition(571818), strand=-1), type='CDS'),
  'CDS'],
 3: [SeqFeature(FeatureLocation(ExactPosition(571818), ExactPosition(571934), strand=-1), type='promoter'),
  'promoter'],
 4: [SeqFeature(FeatureLocation(ExactPosition(571942), ExactPosition(572044), strand=-1), type='misc_RNA'),
  'misc_RNA'],
 5: [SeqFeature(FeatureLocation(ExactPosition(572044), ExactPosition(572066), strand=-1), type='promoter'),
  'promoter']}

In [114]:

for k, v in correspondance(20, positions=set(range(1, 1000000))):
    if int(v.location.start) in cas1:
        ann[]
    else:
        ann = {v.qualifiers['locus_tag'][0]:[int(v.location.start), int(v.location.end), '-'] for k, v in ann.items() if 'locus_tag' in v.qualifiers}
    if positions:
        annotations = correspondance(genome=genome, positions=positions)
        annotations = {}
    corresponde = {'Gene_{}_{}'.format(v[0], v[1]-1):k for k, v in annotations.items()}

SyntaxError: invalid syntax (<ipython-input-114-acf61a0d8854>, line 3)

In [181]:
process_effect(snpcalls1.iloc[102962].EFF, 370436)

{'GAAAAAAAAACTAG': ['frameshift_variant',
  'HIGH',
  'transcript',
  'p.Leu1675fs',
  'gene']}

In [185]:
simplify_effect(snpcalls1)

,SAMPLE,PASS,POS,QUAL,TOT,REFN,ALTN,FRAC,REF,ALT,EFF,IMPACT,AFF,MUT,ANN_TYPE
1,p2_2_S1,2,70,4.927350e-15,60,58,2,3.448276,C,T,upstream_gene_variant,MODIFIER,MPN001,c.-622C>T,intergenic
2,p2_2_S1,2,149,0.000000e+00,75,74,1,1.351351,T,C,upstream_gene_variant,MODIFIER,MPN001,c.-543T>C,intergenic
3,p2_2_S1,2,150,2.263910e-15,69,68,1,1.470588,A,T,upstream_gene_variant,MODIFIER,MPN001,c.-542A>T,intergenic
4,p2_2_S1,2,154,0.000000e+00,62,61,1,1.639344,T,C,upstream_gene_variant,MODIFIER,MPN001,c.-538T>C,intergenic
5,p2_2_S1,2,176,3.420820e-15,58,57,1,1.754386,TAAT,CAAC,upstream_gene_variant,MODIFIER,MPN001,c.-516_-513delTAATinsCAAC,intergenic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
508235,p3IPTG_A_S1b,18,822775,3.631530e-15,171,170,1,0.588235,AACGT,GACGC,missense_variant,MODERATE,MPN688,p.Asn44Ser,gene
508236,p3IPTG_A_S1b,18,822787,1.152910e-15,175,173,2,1.156069,A,G,synonymous_variant,LOW,MPN688,p.Gly41Gly,gene
508237,p3IPTG_A_S1b,18,822830,3.857310e-15,165,164,1,0.609756,T,G,missense_variant,MODERATE,MPN688,p.Lys27Thr,gene
508238,p3IPTG_A_S1b,18,822867,0.000000e+00,172,171,1,0.584795,TTTTT,CTTTC,missense_variant,MODERATE,MPN688,p.Thr15Ala,gene


In [186]:
effect1 = simplify_effect(snpcalls1.head(200))
effect2 = simplify_effect(snpcalls2, genome=20)

In [189]:
effect2.to_csv('./results/effect2.csv', sep='\t')

In [195]:
effect2[effect2['ALTN']>=2]

,SAMPLE,PASS,POS,QUAL,TOT,REFN,ALTN,FRAC,REF,ALT,EFF,IMPACT,AFF,MUT,ANN_TYPE
4,p2_6_S2,2,39,2.676060e-14,228,226,2,0.884956,TGCT,AGCA,upstream_gene_variant,MODIFIER,MPN001,c.-653_-650delTGCTinsAGCA,intergenic
5,p2_6_S2,2,47,0.000000e+00,248,246,2,0.813008,ACGTG,TCGTA,upstream_gene_variant,MODIFIER,MPN001,c.-645_-641delACGTGinsTCGTA,intergenic
8,p2_6_S2,2,71,0.000000e+00,292,290,2,0.689655,CGCGAAAAAGA,TCCATAGTACT,upstream_gene_variant,MODIFIER,MPN001,c.-621_-611delCGCGAAAAAGAinsTCCATAGTACT,intergenic
11,p2_6_S2,2,115,7.127220e-15,323,321,2,0.623053,TTTGCCG,TCAGCCC,upstream_gene_variant,MODIFIER,MPN001,c.-576_-571delTTGCCGinsCAGCCC,intergenic
12,p2_6_S2,2,133,1.401310e-14,343,341,2,0.586510,A,T,upstream_gene_variant,MODIFIER,MPN001,c.-559A>T,intergenic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418219,p15_7_S5,15,829438,1.974090e-15,150,148,2,1.351351,ACAAGC,AGAAGT,missense_variant,MODERATE,MPN688,p.Val26Leu,gene
418221,p15_7_S5,15,829469,1.838480e-15,170,167,3,1.796407,T,C,missense_variant,MODERATE,MPN688,p.Thr16Ala,gene
418222,p15_7_S5,15,829474,0.000000e+00,162,160,2,1.250000,T,C,missense_variant,MODERATE,MPN688,p.Lys14Arg,gene
418224,p15_7_S5,15,829501,0.000000e+00,160,156,4,2.564103,AAAGAAATAAT,GAGGAAATAAT,missense_variant,MODERATE,MPN688,p.Phe5Ser,gene


In [63]:
# High impact mutations in Cas Par in samples C5
effect1.sort_values('FRAC')

,SAMPLE,PASS,POS,QUAL,TOT,REFN,ALTN,FRAC,REF,ALT,EFF,IMPACT,AFF,MUT
1533,p3_D_S2,3,566618,0.000000e+00,985,984,1,0.101626,C,T,missense_variant,MODERATE,LacI4,p.Val21Met
1491,p3_D_S2,3,566174,0.000000e+00,962,961,1,0.104058,T,A,missense_variant,MODERATE,LacI4,p.Asn169Tyr
1532,p3_D_S2,3,566617,1.055200e-15,960,959,1,0.104275,A,G,missense_variant,MODERATE,LacI4,p.Val21Ala
1469,p3_D_S2,3,566002,1.296050e-15,960,959,1,0.104275,G,C,missense_variant,MODERATE,LacI4,p.Ala226Gly
1529,p3_D_S2,3,566608,0.000000e+00,958,957,1,0.104493,T,A,missense_variant,MODERATE,LacI4,p.Tyr24Phe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4176,p3IPTG_A_S1b,18,566416,3.471600e-14,281,255,26,10.196078,GGTGCGT,CGTTCTT,missense_variant,MODERATE,LacI4,p.HisAlaPro86GlnGluArg
4289,p3IPTG_A_S1b,18,569127,8.076710e-15,235,210,25,11.904762,CAAACT,CAACT,frameshift_variant,HIGH,cas9B,p.Phe897fs
3793,p3IPTG_A_S1,18,569125,0.000000e+00,534,463,71,15.334773,ATCAAACT,GTCAACC,frameshift_variant&missense_variant,HIGH,cas9B,p.Lys896fs
4146,p3IPTG_A_S1b,18,566053,0.000000e+00,271,224,47,20.982143,CGCAA,CACAT,missense_variant,MODERATE,LacI4,p.LeuArg208MetCys


In [196]:
plt.figure(figsize=(10,10))
sns.scatterplot(x='POS', y='FRAC', hue='PASS', data=effect1[(effect1['IMPACT']=='HIGH') & (effect1['FRAC']>=1)])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
effect21[effect21['IMPACT']=='HIGH'].sort_values('FRAC')


,SAMPLE,PASS,POS,QUAL,TOT,REFN,ALTN,FRAC,REF,ALT,EFF,IMPACT,AFF,MUT
40,p3_B_S2,3,566670,0.000000e+00,226,225,1,0.444444,CTTTGC,CTTGC,frameshift_variant,HIGH,LacI4,p.Lys3fs
1,p2_6_S2,2,565557,2.712640e-15,155,154,1,0.649351,TCGCT,CCGCC,stop_lost,HIGH,LacI4,p.Ter373Trpext*?
29,p3_B_S1,3,566676,1.994980e-14,226,224,2,0.892857,CATA,TATT,start_lost,HIGH,LacI4,p.Met1?
27,p3_B_S1,3,566634,2.558530e-14,112,111,1,0.900901,AGGCT,AGGGCT,frameshift_variant,HIGH,LacI4,p.Val16fs
58,p15_7_S5,15,567734,5.514750e-15,79,78,1,1.282051,A,T,stop_gained,HIGH,cas9B,p.Leu1362*
10,p2_6_S4,2,565585,1.952790e-14,72,71,1,1.408451,ACTTG,GCTTA,stop_gained,HIGH,LacI4,p.GlnVal364*
52,p15_7_S5,15,565556,0.000000e+00,59,58,1,1.724138,A,T,stop_lost,HIGH,LacI4,p.Ter375Lysext*?
7,p2_6_S4,2,565556,0.000000e+00,97,95,2,2.105263,A,T,stop_lost,HIGH,LacI4,p.Ter375Lysext*?
24,p2_6_S4,2,567751,2.863080e-15,100,97,3,3.092784,ATAAAG,TTAATA,stop_gained,HIGH,cas9B,p.LeuTyr1355*


In [19]:
effect22[effect22['IMPACT']=='HIGH'].sort_values('FRAC')

,SAMPLE,PASS,POS,QUAL,TOT,REFN,ALTN,FRAC,REF,ALT,EFF,IMPACT,AFF,MUT
118,p2_6_S2,2,786095,3.616460e-15,404,403,1,0.248139,T,G,stop_gained,HIGH,cat,p.Tyr176*
130,p2_6_S2,2,786200,0.000000e+00,401,400,1,0.250000,C,G,stop_gained,HIGH,cat,p.Tyr211*
47,p2_6_S2,2,785572,7.669890e-15,336,335,1,0.298507,AGAAAAA,TGTAAT,frameshift_variant&stop_gained,HIGH,cat,p.Glu2fs
54,p2_6_S2,2,785610,1.567030e-15,322,321,1,0.311526,C,T,stop_gained,HIGH,cat,p.Gln15*
55,p2_6_S2,2,785614,1.567030e-15,322,321,1,0.311526,G,A,stop_gained,HIGH,cat,p.Trp16*
59,p2_6_S2,2,785651,0.000000e+00,315,314,1,0.318471,TGCTC,AGCTT,stop_gained,HIGH,cat,p.Gln30*
93,p2_6_S2,2,785881,0.000000e+00,293,292,1,0.342466,TCTG,ACTA,stop_gained,HIGH,cat,p.LeuTrp105*
384,p3_B_S1,3,786189,0.000000e+00,280,279,1,0.358423,TTAC,CTAT,stop_gained,HIGH,cat,p.Gln209*
331,p3_B_S1,3,785573,2.991160e-15,253,252,1,0.396825,GAAAAAAATCAC,GAAAAAAAATCAC,frameshift_variant,HIGH,cat,p.Ile5fs
337,p3_B_S1,3,785643,0.000000e+00,252,251,1,0.398406,C,T,stop_gained,HIGH,cat,p.Gln26*


,POS,REF,ALT,QUAL,TOT,REFN,ALTN,ALTS,FRAC,SAMPLE,P,EFF
0,70,C,T,4.927350e-15,60,58,2,2,3.333333,p2_2_S1,2,T|upstream_gene_variant|MODIFIER|dnaN|MPN001|t...
1,149,T,C,0.000000e+00,75,74,1,1,1.333333,p2_2_S1,2,C|upstream_gene_variant|MODIFIER|dnaN|MPN001|t...
2,150,A,T,2.263910e-15,69,68,1,1,1.449275,p2_2_S1,2,T|upstream_gene_variant|MODIFIER|dnaN|MPN001|t...
3,154,T,C,0.000000e+00,62,61,1,1,1.612903,p2_2_S1,2,C|upstream_gene_variant|MODIFIER|dnaN|MPN001|t...
4,176,TAAT,CAAC,3.420820e-15,58,57,1,1,1.724138,p2_2_S1,2,CAAC|upstream_gene_variant|MODIFIER|dnaN|MPN00...
...,...,...,...,...,...,...,...,...,...,...,...,...
508435,822775,AACGT,GACGC,3.631530e-15,171,170,1,1,0.584795,p3IPTG_A_S1b,18,GACGC|missense_variant|MODERATE|soj|MPN688|tra...
508436,822787,A,G,1.152910e-15,175,173,2,2,1.142857,p3IPTG_A_S1b,18,G|synonymous_variant|LOW|soj|MPN688|transcript...
508437,822830,T,G,3.857310e-15,165,164,1,1,0.606061,p3IPTG_A_S1b,18,G|missense_variant|MODERATE|soj|MPN688|transcr...
508438,822867,TTTTT,CTTTC,0.000000e+00,172,171,1,1,0.581395,p3IPTG_A_S1b,18,CTTTC|missense_variant|MODERATE|soj|MPN688|tra...


In [80]:
plt.figure()
effect21['logFRAC'] = np.log2(effect21['FRAC'])
sns.violinplot(x='IMPACT', y='FRAC', hue='PASS', data=effect1)

/home/smiravet/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [42]:
snpcalls1[snpcalls1['SAMPLE']=='p2_2_S1'].sort_values('FRAC')

,POS,REF,ALT,QUAL,TOT,REFN,ALTN,ALTS,FRAC,SAMPLE,P,EFF
18482,566325,C,T,1.469330e-15,205,204,1,1,0.487805,p2_2_S1,2,T|synonymous_variant|LOW|Gene_565549_566677|Ge...
18481,566311,A,G,7.046470e-16,198,197,1,1,0.505051,p2_2_S1,2,G|missense_variant|MODERATE|Gene_565549_566677...
18485,566383,GCC,CCT,0.000000e+00,181,180,1,1,0.552486,p2_2_S1,2,CCT|missense_variant|MODERATE|Gene_565549_5666...
18483,566361,A,G,0.000000e+00,172,171,1,1,0.581395,p2_2_S1,2,G|synonymous_variant|LOW|Gene_565549_566677|Ge...
18476,566244,A,G,1.167980e-15,171,170,1,1,0.584795,p2_2_S1,2,G|synonymous_variant|LOW|Gene_565549_566677|Ge...
...,...,...,...,...,...,...,...,...,...,...,...,...
6416,195428,CAAAAAAAAAAAAAAAGTAAAATAGAAAAGC,"CAAAATAAGAAAAAAAGTAAAATAGAAAAGC,CAAAAAAAAAAAAA...",0.000000e+00,24,15,9,"1,1,1,6",37.500000,p2_2_S1,2,CAAAATAAGAAAAAAAGTAAAATAGAAAAGC|upstream_gene_...
17103,528761,GTTTTTTTTTTTTTTTTTGAAAGA,"GTTTTTTTTTTTTTTTTGAAAGA,GTTTTTTTTTTTTTTTTTTGAAAGA",0.000000e+00,28,16,12,"6,6",42.857143,p2_2_S1,2,GTTTTTTTTTTTTTTTTGAAAGA|upstream_gene_variant|...
20563,629173,GTTTTTTTTTTTTTTTTAGTTTGAAC,"GTTTTTTTTTATTTATAAGTTTGAAC,GTTTTTTTTTTTTTTTAGT...",1.383230e-14,30,17,13,"1,2,6,2,2",43.333333,p2_2_S1,2,GTTTTTTTTTATTTATAAGTTTGAAC|upstream_gene_varia...
4753,141270,CAGAGAGAGAGAGAGAGAGAGC,"CAGAGAGAGAGAGAGAGAGC,CAGAGAGAGAGAGAGAGAGAGAGC",0.000000e+00,52,27,25,"7,18",48.076923,p2_2_S1,2,CAGAGAGAGAGAGAGAGAGC|upstream_gene_variant|MOD...


In [47]:
simplify_effect(snpcalls1, genome=20)

UnboundLocalError: local variable 'annotations' referenced before assignment

In [45]:
# Figure


def plot_analysis(df):
    repro = [k for k, v in Counter(list(df.POS)).items() if v>=2]
    plt.figure()
    plt.subplot(3,3,1)
    sns.lineplot(x='P', y='FRAC', data=df[(df['POS'].isin(posE)) & (df['POS'].isin(repro))], label='E')
    sns.lineplot(x='P', y='FRAC', data=df[(df['POS'].isin(posN)) & (df['POS'].isin(repro))], label='NE')
    sns.lineplot(x='P', y='FRAC', data=df[(df['POS'].isin(cas1)) & (df['POS'].isin(repro))], label='C')


plot_analysis(snpcalls1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 4. Substituion rate per base comparing cassette versus other distributions

To evaluate the selection in the cassette we compare the rate of substitutions per base occuring within the cassette, a set of essential and non-essential genes and with the general distribution observed at genome level. In this case we do not care about the fraction each variant is found but how many times a variant is found in the cassette versus the other distributions.

In [11]:
# Show the ratio of mutation in cassette versus general
def mutation_rate(snpdf, segment, mintot=5, minrefn=3, minaltn=2):
    _df = snpdf[(snpdf['TOT']>=mintot) & (snpdf['REFN']>=minrefn) & (snpdf['ALTN']>=minaltn)].copy()    
    rs = {}
    for sample in set(_df.SAMPLE):
        total_len = 816394
        _df2 = _df[_df['SAMPLE']==sample].copy()
        NE = _df2[_df2['POS'].isin(posN)]
        ES = _df2[_df2['POS'].isin(posE)]
        TG = _df2[~_df2['POS'].isin(segment)]
        CS = _df2[_df2['POS'].isin(segment)]
        #total_len+=len(segment)
        rs[sample] = [sample, sample.split('_')[0],
                      int(sample.split('_')[0].replace('p', '').replace('3IPTG', '18')),
                      TG.shape[0], ES.shape[0], NE.shape[0], CS.shape[0], 
                      sum(TG['FRAC']), sum(ES['FRAC']), sum(NE['FRAC']), sum(CS['FRAC'])]
    jj = pd.DataFrame.from_dict(rs, orient='index')
    jj.columns = ['SAMPLE', 'COND', 'P', 'T', 'E', 'N', 'C', 'TF', 'EF', 'NF', 'CF']
    jj['T%'] = 100*jj['T']/(total_len)  
    jj['E%'] = 100*jj['E']/len(posE)
    jj['N%'] = 100*jj['N']/len(posN)
    jj['C%'] = 100*jj['C']/len(segment)
    return jj

def tp(df, order):
    rs = {}
    c=0
    for row, col in df.iterrows():
        rs[c] = [col['COND'],'Essential', col['E%'], col['EF']]
        rs[c+1] = [col['COND'],'Non-essential', col['N%'], col['NF']]
        rs[c+2] = [col['COND'],'Chromosome', col['T%'], col['TF']]
        rs[c+3] = [col['COND'],'Cassette', col['C%'], col['CF']]
        c+=4
    a = pd.DataFrame.from_dict(rs, orient='index')
    a.columns = ['Condition', 'Loci', 'Variation per base [%]', 'Accumulated Fraction [%]']
    return orderdf(a, ordered_classes=order, col='Condition')

In [32]:
mutrate11 = mutation_rate(snpcalls1, segment=cas1)
mutrate21 = mutation_rate(snpcalls2, segment=cas1)
mutrate22 = mutation_rate(snpcalls2, segment=cas2)
mutrate212 = mutation_rate(snpcalls2, segment=cas12)

In [33]:
plt.close('all')
plt.figure(figsize=(15, 3))
c = 1
for text, df in zip(['C5 Par', 'C20 Par', 'C20 3b'], [mutrate11, mutrate21, mutrate22]):
    plt.subplot(1,3,c)
    if text=='C5 Par':
        sns.barplot(x='Condition', y='Variation per base [%]', hue='Loci', palette='mako', data=tp(df, ['p2','p3','p15', 'p3IPTG']))
    else:
        sns.barplot(x='Condition', y='Variation per base [%]', hue='Loci', palette='mako', data=tp(df, ['p2','p3','p15']))
    plt.ylim(0, 15)
    c+=1
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:
from dnds_functions import evolution_of_sample, dnds
ttt = evolution_of_sample(effect1[effect1['IMPACT']=='HIGH'])

In [40]:
effectC= simplify_effect(snpcalls1, cas1, minaltn=3)
effectE= simplify_effect(snpcalls1, posE, minaltn=3)
effectN= simplify_effect(snpcalls1, posN, minaltn=3)

tttC = evolution_of_sample(effectC)
tttE = evolution_of_sample(effectE)
tttN = evolution_of_sample(effectN)

In [53]:
tttE

{'p15_F_S1': [0.015,
  0.011,
  1.3636363636363638,
  0.015,
  0.011,
  1.3636363636363638],
 'p3IPTG_A_S1': [0.418,
  0.265,
  1.5773584905660376,
  0.612,
  0.327,
  1.8715596330275228],
 'p3_D_S3': [0.012, 0.0, '+', 0.012, 0.0, '+'],
 'p2_2_S1': [0, 0, '/', 0, 0, '/'],
 'p3IPTG_A_S1b': [0, 0, '/', 0, 0, '/'],
 'p15_F_S5': [0, 0, '/', 0, 0, '/'],
 'p2_2_S3': [0.009000000000000001,
  0.022,
  0.40909090909090917,
  0.009000000000000001,
  0.022,
  0.40909090909090917],
 'p2_2_S2': [0.006, 0.0, '+', 0.006, 0.0, '+'],
 'p3_D_S2': [0.03,
  0.06599999999999999,
  0.4545454545454546,
  0.03,
  0.06599999999999999,
  0.4545454545454546]}

In [55]:
plt.figure()
x, y = [[], []]
for k, v in tttC.items():
    if type(v[-1])==float and type(tttE[k][-1])==float:
        x.append(tttE[k][-1])
        y.append(v[-1])
plt.scatter(x, y)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [307]:
plt.figure()
plt.subplot(2,2,1)
sns.countplot(x='SAMPLE', data=snpcalls1)
plt.xticks(rotation=45, ha='right')
plt.title('Variant count per sample')

plt.subplot(2,2,2)
sns.boxplot(x='SAMPLE', y='FRAC', data=snpcalls1)
plt.xticks(rotation=45, ha='right')
plt.title('Fraction distribution per sample')

plt.subplot(2,2,3)
sns.countplot(x='SAMPLE', data=snpcalls2)
plt.xticks(rotation=45, ha='right')
plt.title('Variant count per sample')

plt.subplot(2,2,4)
sns.boxplot(x='SAMPLE', y='FRAC', data=snpcalls2)
plt.xticks(rotation=45, ha='right')
plt.title('Fraction distribution per sample')

plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

General exploration of the most representative variants:

In [308]:

plt.figure(figsize=(15, 10))
plt.subplot(3,1,1)
sns.scatterplot(x='POS', y='FRAC', hue='SAMPLE', data=snpcalls1[(snpcalls1['FRAC']>=5) & (snpcalls1['TOT']>=10) & (snpcalls1['P']<=15) & (snpcalls1['FRAC']<100)])
plt.axvspan(565510, 572114, color='gray', alpha=0.2)
plt.xlabel('')
plt.ylabel('Fraction of variant reads [%]')
plt.xlim(1, 890000)

plt.subplot(3,1,2)
sns.scatterplot(x='POS', y='FRAC', hue='SAMPLE', data=snpcalls1[(snpcalls1['FRAC']>=5) & (snpcalls1['TOT']>=10) & (snpcalls1['P']>15) & (snpcalls1['FRAC']<100)])
plt.axvspan(565510, 572114, color='gray', alpha=0.2)
plt.xlabel('')
plt.ylabel('Fraction of variant reads [%]')
plt.xlim(1, 890000)

plt.subplot(3,1,3)
sns.scatterplot(x='POS', y='FRAC', hue='SAMPLE', data=snpcalls2[(snpcalls2['FRAC']>=5) & (snpcalls2['TOT']>=10) & (snpcalls2['FRAC']<100)])
plt.axvspan(cass_par[0], cass_par[1], color='gray', alpha=0.2)
plt.axvspan(cass_3b[0], cass_3b[1], color='gray', alpha=0.2)
plt.xlabel('genome position [bp]')
plt.ylabel('Fraction of variant reads [%]')
plt.xlim(1, 890000)

plt.savefig('./figures/supfig1.svg')
plt.savefig('./figures/supfig1.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Generate final snp working data


In [17]:
cass_par = [565510, 572124]
cass_3b = [780850, 787445]

all_f1_par = evaluate_cassete_mut(snpcalls1, sten=cass_par)
all_f2_par = evaluate_cassete_mut(snpcalls2, sten=cass_par)
all_f2_3b = evaluate_cassete_mut(snpcalls2, sten=cass_3b)

NameError: name 'evaluate_cassete_mut' is not defined

In [18]:
all_f2_3b

NameError: name 'all_f2_3b' is not defined

# Selection of variants
In this case we evaluate the fraction of each variant and how it is fixed along passages. As not all the samples are directly related, we consider the induced sample (IPTG) as 18. In this sense, if a mutation found in passages 2, 3 or 15 is found in 18 with a higher rate it implies that this variants are able to confere the capability to surpass the killswitch

In [19]:
def plot_evolution(df):
    positions = set(df[df['FRAC']>1].POS)
    plt.figure()
    plt.subplot(1,2,1)
    sns.lineplot(x='P', y='FRAC', hue='POS', data=df[df['POS'].isin(positions)])
    plt.subplot(1,2,2)
    sns.boxplot(x='P', y='FRAC', data=df[df['POS'].isin(positions)])


In [20]:
plot_evolution(all_f1_par)


NameError: name 'all_f1_par' is not defined

In [21]:
plot_evolution(all_f2_par)

NameError: name 'all_f2_par' is not defined

In [22]:
plot_evolution(all_f2_3b)

NameError: name 'all_f2_3b' is not defined

In [23]:
genome_seq2 = u.load_genome('./data/C20_refSeq.gb')

NameError: name 'u' is not defined

In [24]:
all_f2_3b[all_f2_3b['FRAC']>=100]

NameError: name 'all_f2_3b' is not defined

In [25]:
'CCCCCCCGGATCCCTCG' in genome_seq2

NameError: name 'genome_seq2' is not defined

In [26]:
selected = all_f1[(all_f1['FRAC']>=2.5) & (all_f1['P']==18)]
selected

NameError: name 'all_f1' is not defined

In [27]:
sum(selected['FRAC'])

NameError: name 'selected' is not defined

The mutations found affect Cas9B and LacI4

In [28]:
[i.split(',') for i in list(selected['EFF'])]


NameError: name 'selected' is not defined

In [29]:
annotations

NameError: name 'annotations' is not defined

In [328]:
(566678-565549+1)/3

376.6666666666667

In [182]:
annotations

{'LacI4': [565549, 566678, '-'],
 'pS': [566678, 566721, '-'],
 'Par cassette': [566747, 567674, '-'],
 'cas9B': [567711, 571818, '-'],
 'pG64': [571818, 571934, '-'],
 'gRNA2': [571942, 572044, '-'],
 'protospacer12 (10 targets)': [572024, 572044, '-'],
 'p438': [572044, 572066, '-'],
 'IR-OR': [572088, 572114, '-'],
 'MTn insertion point': [572113, 572124, '-']}

In [330]:
from Bio.Seq import Seq
genes_affected = {}
for k, v in annotations.items():
    if k in ['cas9B', 'LacI4']:
        if k=='cas9B':
            seq = Seq(genome_C5[v[0]:v[1]])
        else:
            seq = Seq(genome_C5[v[0]:v[1]])
        print(len(seq)/3)
        rvc = seq.reverse_complement()
        mrn = rvc.transcribe()
        prt = mrn.translate(table=4)
        genes_affected[k] = [seq, rvc, mrn, prt, len(prt)]

376.3333333333333
1369.0


/home/smiravet/.local/lib/python3.6/site-packages/Bio/Seq.py:2309: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning)


In [333]:
genes_affected['LacI4'][3][-10:]

Seq('RLESGQ*R*V', HasStopCodon(ExtendedIUPACProtein(), '*'))

LacI4 has extra bases in the annotation, that makes it to be non-codonic. I will consider the first stop found (move the end 10 bases)...

In [33]:
from Bio.Seq import Seq
genes_affected = {}
for k, v in annotations.items():
    if k in ['cas9B', 'LacI4']:
        if k=='cas9B':
            seq = Seq(genome_seq[v[0]:v[1]])
        else:
            seq = Seq(genome_seq[v[0]+10:v[1]])
        print(len(seq)/3)
        rvc = seq.reverse_complement()
        mrn = rvc.transcribe()
        prt = mrn.translate(table=4)
        genes_affected[k] = [seq, rvc, mrn, prt, len(prt)]

NameError: name 'annotations' is not defined

In [34]:
genes_affected['LacI4'][3][-10:]

KeyError: 'LacI4'

In [ ]:
aa

# Evaluate effect of the selected mutations

In [35]:
snpcalls1

,POS,REF,ALT,QUAL,TOT,REFN,ALTN,FRAC,SAMPLE,P,EFF
0,70,C,T,4.927350e-15,60,58,2,3.333333,p2_2_S1,2,T|upstream_gene_variant|MODIFIER|dnaN|MPN001|t...
1,149,T,C,0.000000e+00,75,74,1,1.333333,p2_2_S1,2,C|upstream_gene_variant|MODIFIER|dnaN|MPN001|t...
2,150,A,T,2.263910e-15,69,68,1,1.449275,p2_2_S1,2,T|upstream_gene_variant|MODIFIER|dnaN|MPN001|t...
3,154,T,C,0.000000e+00,62,61,1,1.612903,p2_2_S1,2,C|upstream_gene_variant|MODIFIER|dnaN|MPN001|t...
4,176,TAAT,CAAC,3.420820e-15,58,57,1,1.724138,p2_2_S1,2,CAAC|upstream_gene_variant|MODIFIER|dnaN|MPN00...
...,...,...,...,...,...,...,...,...,...,...,...
41166,822775,AACGT,GACGC,3.631530e-15,171,170,1,0.584795,p3IPTG_A_S1b,18,GACGC|missense_variant|MODERATE|soj|MPN688|tra...
41167,822787,A,G,1.152910e-15,175,173,2,1.142857,p3IPTG_A_S1b,18,G|synonymous_variant|LOW|soj|MPN688|transcript...
41168,822830,T,G,3.857310e-15,165,164,1,0.606061,p3IPTG_A_S1b,18,G|missense_variant|MODERATE|soj|MPN688|transcr...
41169,822867,TTTTT,CTTTC,0.000000e+00,172,171,1,0.581395,p3IPTG_A_S1b,18,CTTTC|missense_variant|MODERATE|soj|MPN688|tra...


NameError: name 'selected' is not defined

In [184]:
selected

NameError: name 'selected' is not defined

In [38]:
selected_ann

NameError: name 'selected_ann' is not defined

In [39]:
selected_ann[selected_ann['IMPACT']=='HIGH']

NameError: name 'selected_ann' is not defined

# Mutation rate study



In [92]:
plt.close('all')

In [44]:
pd.melt(mutrate1.sort_values('P')[[i for i in mutrate1.columns if '%' in i]])

NameError: name 'mutrate1' is not defined

In [45]:
plt.figure()
plt.boxplot([mutrate1['N%'], mutrate2['N%']])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

NameError: name 'mutrate1' is not defined

In [46]:
plt.close('all')
plt.figure()
x, y = ['OUT%', 'IN%']
plt.plot([0,16],[0,16], c='grey', linestyle='--')
sns.scatterplot(x=x, y=y, data=mutrate1)
for i in range(mutrate1.shape[0]):
    plt.text(x=mutrate1[x][i]+0.2,y=mutrate1[y][i]+0.3,s=mutrate1.index[i],
             fontdict=dict(size=8))
sns.scatterplot(x=x, y=y, data=mutrate2)
c=0
for i in range(mutrate2.shape[0]):

    if c==0:
        plt.text(x=mutrate2[x][i]+0.2,y=mutrate2[y][i]+0.4,s=mutrate2.index[i],
                 fontdict=dict(size=10))
        c==1
    else:
        plt.text(x=mutrate2[x][i]+0.2,y=mutrate2[y][i]+0.2,s=mutrate2.index[i],
                 fontdict=dict(size=8))
        c==0
plt.xlabel('Genome Variant Rate [%]')
plt.ylabel('Cassette Variant Rate [%]')
plt.xlim(0,16)
plt.ylim(0,16)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

NameError: name 'mutrate1' is not defined

In [47]:
from scipy.stats import wilcoxon, ttest_rel
from collections import Counter

wrs = {}
trs = {}
ns = dict(Counter(mutrate.P))
for p in set(mutrate.P):
    wrs[p] = wilcoxon(x=list(mutrate[mutrate['P']==p]['OUT%']),
                      y=list(mutrate[mutrate['P']==p]['IN%']))[1]
    trs[p] = ttest_rel(list(mutrate[mutrate['P']==p]['OUT%']),
                       list(mutrate[mutrate['P']==p]['IN%']))[1]

NameError: name 'mutrate' is not defined

In [48]:
ns, wrs, trs

NameError: name 'ns' is not defined

null hypothesis (same mut rate) cannot be rejected at a confidence level of 5%, only at early passages 2 and 3 seems to be limited

In [49]:
snpcalls50

NameError: name 'snpcalls50' is not defined

# Plot variations

In [50]:
cass_par

[565510, 572124]

In [51]:
selected

NameError: name 'selected' is not defined

In [52]:

from reportlab.lib import colors
from reportlab.lib.units import cm
from Bio.Graphics import GenomeDiagram


def plot_variations(genbank_file, coords, variations):
    record = SeqIO.read(genbank_file, "genbank")

    gd_diagram = GenomeDiagram.Diagram(record.id)
    gd_track_for_features = gd_diagram.new_track(1, name="Annotated Features")
    gd_feature_set = gd_track_for_features.new_set()

    for feature in record.features:
        if feature.location.start>=coords[0] and feature.location.start<=coords[1] and feature.type!='Polymorphism':
            print(feature.type, feature.qualifiers['label'], feature.location)
            feature.location.strand=-1
            #if feature.type != "gene":
                # Exclude this feature
            #    continue
            #if len(gd_feature_set) % 2 == 0:
            #    color = colors.blue
            if feature.type=='CDS':
                color = colors.lightblue
                gd_feature_set.add_feature(
                    feature, sigil="ARROW", arrowshaft_height=1.0, color=color, height=0.2, label=True, label_size=14, label_angle=0
                )
            # else:
            #     color = colors.lightblue
            #     gd_feature_set.add_feature(
            #         feature, sigil="BOX", color=color, height=0.1, label=True, label_size=14, label_angle=10)
    colordic = {'stop': colors.firebrick, 'frameshift':colors.red, 'missense':colors.orange}
    done = []

    for pos, eff in zip(variations.POS, variations.EFF):
        if pos not in done:
            effi = eff.split('|')[1].split('_')[0]
            color = colordic[effi]
            feature = SeqFeature(FeatureLocation(pos, pos+1))
            gd_feature_set.add_feature(
                    feature,
                    color=color,
                    name=' '+effi,
                    label=True,
                    label_size=15,
                    size=20
                    #label_color=color,
                )
            done.append(pos)
            for i in range(pos-30, pos+30):
                done.append(i)
    gd_diagram.draw(format="linear", pagesize="A5", fragments=1, start=565510, end=572124)
    gd_diagram.write("plasmid_linear_nice.svg", "SVG")

In [53]:
plot_variations('./data/C5_refSeq.gb', coords=[565549,571818], variations=selected)

NameError: name 'selected' is not defined

In [637]:
raw_seq, new_seq = get_new_sequence(seq_record5, 'LacI4', pos=[565576, 'AGGC', 'GGGT'])
yyy = dnds(raw_seq, new_seq)

268.6666666666668


In [648]:
effect1[effect1['POS']==571815]

,SAMPLE,PASS,POS,QUAL,TOT,REFN,ALTN,FRAC,REF,ALT,EFF,IMPACT,AFF,MUT
2140,p3_D_S2,3,571815,0.0,674,668,6,0.898204,CCAGATGTAA,TCTGTTGTTT,start_lost,HIGH,cas9B,p.LeuAsp1?


In [656]:
for k, v in ttt.items():
    print(k, v[-1])

p3IPTG_A_S1 2.404754044239023
p3_D_S2 1.7427616926503313
p15_F_S1 2.3935483870967738
p15_F_S5 3.4130506790327924
p3IPTG_A_S1b 2.6608581600149894
p3_D_S3 2.117820324005891
p2_2_S3 2.3640178337267255
p2_2_S1 2.127371273712737
p2_2_S2 1.7000000000000002


In [56]:
df = pd.read_csv(, header=60, sep='\t')
df2 = df[(df['INFO'].str.contains('CIGAR=1X;'))].sample(n=150, random_state=1).copy()

SyntaxError: invalid syntax (<ipython-input-56-9a8f373c509c>, line 1)

In [57]:
pn, ps, pns, dn, ds, dns = 0, 0, 0, 0, 0, 0
for item in sorted(output[chromosome]):
    raw_seq, new_seq = get_new_sequence(args.reference, chromosome, output[chromosome][item][0].qualifiers[
        'locus_tag'][0], output[chromosome][item][1])
    newpn, newps, newdn, newds = dnds(raw_seq, new_seq)
    pn += newpn
    ps += newps
    dn += newdn
    ds += newds
if pn == 0 and ps == 0:
    pns, dns = '/', '/'
elif ps == 0:
    pns, dns = '+', '+'
elif pn == 0:
    pns, dns = '-', '-'
else:
    pns, dns = pn / ps, dn / ds
print args.query, chromosome, pn, ps, pns, dn, ds, dns

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(args.query, chromosome, pn, ps, pns, dn, ds, dns)? (<ipython-input-57-012244273d6f>, line 18)

In [58]:
for records in SeqIO.parse(, "genbank"):
    print(/)
    raw_seq, new_seq = get_new_sequence('./data/C5_refSeq.gb', 'C5', output[chromosome][item][0].qualifiers[
        'locus_tag'][0], output[chromosome][item][1])

SyntaxError: invalid syntax (<ipython-input-58-9fd2afaa1715>, line 1)